# Notebook exploring tool usage

In [1]:
import os
from dotenv import load_dotenv

while 'notebooks' in os.getcwd():
    os.chdir('..')

load_dotenv('.env')

True

In [22]:
import json
import random
from typing import Union

from IPython.display import display, Markdown

from openai import OpenAI

## Raw LLM

In [23]:
class LLM:
    def __init__(self, model: str = 'gpt-4.1'):
        self.client = OpenAI()
        self.model = model
    
        
    def __call__(self, messages: Union[str, list[dict[str, str]]]):
        if isinstance(messages, str):
            messages = [{'role': 'user', 'content': messages}]
        result = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
        )
        return result.choices[0].message.content

In [4]:
llm = LLM()

In [5]:
llm('What up')

'Hey! Not much—just here, ready to help. What’s up with you?'

## Web search (built-in openai tool)

In [6]:
%%time
response = llm.client.responses.create(
    model=llm.model,
    tools=[{"type": "web_search_preview"}],
    input="When is the next UFC?"
)

CPU times: user 250 ms, sys: 78.1 ms, total: 328 ms
Wall time: 7.1 s


In [7]:
response.usage.model_dump()

{'input_tokens': 307,
 'input_tokens_details': {'cached_tokens': 0},
 'output_tokens': 187,
 'output_tokens_details': {'reasoning_tokens': 0},
 'total_tokens': 494}

In [8]:
print(json.dumps(response.model_dump(), indent=3, ensure_ascii=False))

{
   "id": "resp_68886104823881a1a96bd71cef076c430d084b7b4220189a",
   "created_at": 1753768196.0,
   "error": null,
   "incomplete_details": null,
   "instructions": null,
   "metadata": {},
   "model": "gpt-4.1-2025-04-14",
   "object": "response",
   "output": [
      {
         "id": "ws_68886105277c81a1a431e017ef6d8f160d084b7b4220189a",
         "action": {
            "query": "next UFC event date",
            "type": "search"
         },
         "status": "completed",
         "type": "web_search_call"
      },
      {
         "id": "msg_68886106f10c81a195113468742c36180d084b7b4220189a",
         "content": [
            {
               "annotations": [
                  {
                     "end_index": 297,
                     "start_index": 242,
                     "title": "Upcoming Events | UFC News",
                     "type": "url_citation",
                     "url": "https://ufc.news/events?utm_source=openai"
                  }
               ],
            

In [9]:
response.output[0]

ResponseFunctionWebSearch(id='ws_68886105277c81a1a431e017ef6d8f160d084b7b4220189a', action=ActionSearch(query='next UFC event date', type='search'), status='completed', type='web_search_call')

In [10]:
response.output[1]

ResponseOutputMessage(id='msg_68886106f10c81a195113468742c36180d084b7b4220189a', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=297, start_index=242, title='Upcoming Events | UFC News', type='url_citation', url='https://ufc.news/events?utm_source=openai')], text='The next UFC event is scheduled for Saturday, August 2, 2025, at the UFC Apex in Las Vegas, Nevada. This event, titled "UFC on ESPN 71: Albazi vs. Taira," will feature a flyweight bout between Amir Albazi and Tatsuro Taira as the main event. ([ufc.news](https://ufc.news/events?utm_source=openai))\n\nThe main card is set to begin at 10:00 PM Eastern Time (ET). For viewers in the United States, the event will be broadcast live on ESPN.\n\nThe UFC Apex is located in Las Vegas, Nevada, and serves as a primary venue for many UFC Fight Night events.\n\nFor the most up-to-date information on the fight card and any potential changes, you can visit the official UFC website or their social media channels. ', ty

In [11]:
display(Markdown(response.output[1].content[0].text))

The next UFC event is scheduled for Saturday, August 2, 2025, at the UFC Apex in Las Vegas, Nevada. This event, titled "UFC on ESPN 71: Albazi vs. Taira," will feature a flyweight bout between Amir Albazi and Tatsuro Taira as the main event. ([ufc.news](https://ufc.news/events?utm_source=openai))

The main card is set to begin at 10:00 PM Eastern Time (ET). For viewers in the United States, the event will be broadcast live on ESPN.

The UFC Apex is located in Las Vegas, Nevada, and serves as a primary venue for many UFC Fight Night events.

For the most up-to-date information on the fight card and any potential changes, you can visit the official UFC website or their social media channels. 

In [12]:
response.output[1].content[0].annotations

[AnnotationURLCitation(end_index=297, start_index=242, title='Upcoming Events | UFC News', type='url_citation', url='https://ufc.news/events?utm_source=openai')]

In [13]:
tool = response.tools[0]
type(tool)

openai.types.responses.web_search_tool.WebSearchTool

## Function calling tool
Probably the most useful tool of letting LLM to decide to use your own function.

In [14]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

In [15]:
%%time
response = llm.client.responses.create(
    model=llm.model,
    tools=tools,
    input="What is the weather in Warsaw, Poland"
)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 566 ms


In [16]:
len(response.output)

1

In [17]:
function_call_response = response.output[0]
function_call_response

ResponseFunctionToolCall(arguments='{"location":"Warsaw, Poland"}', call_id='call_i6RGcKGZvDLtRWYkEU9yB3Gw', name='get_weather', type='function_call', id='fc_6888610a1fec81a1a6b0cc66f35b5faf0c0b13f7b2e8eba3', status='completed')

In [18]:
def get_weather(location: str):
    return f"Weather in {location} is {random.randint(0, 40)} °C"

In [19]:
functions = {
    'get_weather': get_weather
}

In [20]:
function_call_response.arguments

'{"location":"Warsaw, Poland"}'

In [21]:
functions[function_call_response.name](**json.loads(function_call_response.arguments))

'Weather in Warsaw, Poland is 7 °C'